In [ ]:
# Instalações das bibliotecas:

#Bibliotecas necessárias para o projeto, como o PyCaret e o SHAP.


!pip install pycaret -q

!pip install shap


In [ ]:
# Importação das bibliotecas:

# Bibliotecas necessárias para trabalhar com os dados, visualizações, modelagem e avaliação de desempenho.

# Algumas das bibliotecas incluídas são:
# Pandas, NumPy, Matplotlib, Seaborn, Scikit-learn, Imbalanced-learn (para o SMOTE), Yellowbrick e PyCaret.


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from yellowbrick.classifier import ROCAUC, ConfusionMatrix
from pycaret.classification import *
import shap


In [ ]:
# Carregar e limpar os dados:
# Definiu a função `load_and_clean_data` para carregar e limpar os dados.
# Essa função lê o arquivo CSV e remove algumas colunas irrelevantes e realiza algumas operações de limpeza nos dados, como calcular a idade dos clientes e o tempo de cadastro.


def load_and_clean_data(file_path):
    df = pd.read_csv(file_path, encoding='utf-8')
    df.drop(['ID', 'Z_CostContact', 'Z_Revenue'], axis=1, inplace=True)
    df.dropna(inplace=True)

    df['Age'] = datetime.now().year - df['Year_Birth']
    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
    df['Time_Customer'] = (datetime.now().date() - df['Dt_Customer'].dt.date) / timedelta(days=365)
    df.drop('Dt_Customer', axis=1, inplace=True)

    return df


In [ ]:
# Visualização de EDA:
# Definiu a função `eda_visualization` para realizar uma análise exploratória dos dados (Exploratory Data Analysis - EDA).
# Nessa função, é criado um pairplot com o Seaborn para visualizar as relações entre as variáveis, com a variável alvo (Response) como hue.


def eda_visualization(df):
    sns.set(style="ticks", color_codes=True)
    sns.pairplot(df, hue="Response")
    plt.show()


In [ ]:
# Preparação do dataset para Modelagem:
# Definiu a função `prepare_data_for_modeling` para preparar os dados para modelagem.
# Nessa função, os dados são separados em variáveis independentes (X) e variável alvo (y), as variáveis categóricas são convertidas em:
# Variáveis dummy e os dados são divididos em conjuntos de treinamento e teste.


def prepare_data_for_modeling(df):
    X = df.drop('Response', axis=1)
    y = df['Response']
    X = pd.get_dummies(X, drop_first=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    y_train = y_train.astype('int')
    y_test = y_test.astype('int')

    return X_train, X_test, y_train, y_test


In [ ]:
# Modelagem com Scikit-learn:

# Definiu a função `train_decision_tree` para treinar um modelo de árvore de decisão utilizando o Scikit-learn.
# Esse modelo é treinado no conjunto de treinamento fornecido como entrada para a função.


def train_decision_tree(X_train, y_train):
    mdl_dt = DecisionTreeClassifier(random_state=42)
    mdl_dt.fit(X_train, y_train)
    return mdl_dt


In [ ]:
# Balanceamento por SMOTE:
# Definiu a função `oversample_with_smote` para aplicar a técnica SMOTE de balanceamento de classes no conjunto de treinamento.
# Essa função utiliza a biblioteca Imbalanced-learn para realizar o balanceamento.

def oversample_with_smote(X_train, y_train):
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train)
    return X_res, y_res


In [ ]:
# Função para plotar a matriz de confusão:

def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
# Função para plotar a curva ROC:

def plot_roc_curve(X_train, y_train, X_test, y_test):
    visualizer = ROCAUC(mdl_dt, classes=[0, 1], micro=False, macro=False)
    visualizer.fit(X_train, y_train)
    visualizer.score(X_test, y_test)
    visualizer.show()


In [ ]:
# Modelagem com PyCaret
def perform_pycaret_modeling(df):
    exp_clf101 = setup(data=df, target='Response', ignore_features=['ID', 'Z_CostContact', 'Z_Revenue'], session_id=42)
    best_model = compare_models()
    dt = create_model('dt')  # Atribui o modelo a uma variável
    interpret_model(dt)   # Interpreta o modelo



In [ ]:
# Carregar e limpar os dados:

df = load_and_clean_data('data.csv')


In [ ]:
# Visualização de EDA:

eda_visualization(df)


In [ ]:
# Preparação do dataset para Modelagem:

X_train, X_test, y_train, y_test = prepare_data_for_modeling(df)


In [ ]:
# Modelagem com PyCaret:

perform_pycaret_modeling(df)

In [ ]:
# Modelagem com Scikit-learn:

mdl_dt = train_decision_tree(X_train, y_train)
ypred_dt = mdl_dt.predict(X_test)
plot_confusion_matrix(y_test, ypred_dt)
plot_roc_curve(X_train, y_train, X_test, y_test)

In [ ]:
# Balanceamento por SMOTE:

X_res, y_res = oversample_with_smote(X_train, y_train)
mdl_dt = train_decision_tree(X_res, y_res)
ypred_dt = mdl_dt.predict(X_test)
plot_confusion_matrix(y_test, ypred_dt)
plot_roc_curve(X_res, y_res, X_test, y_test)
